<a href="https://colab.research.google.com/github/lorek/Probability_Course_Labs/blob/main/LAB_LIST_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(Click above to open the notebook in Google Colab. Then save it to your Google Drive)

# Laboratorium z rachunku prawdopodobieństwa
# Lista 4

#Rozgrzewka: konsekwentny hazardzista

Pewien gracz codziennie przychodzi do kasyna, za każdym razem z kwotą 1000 dolarów. Gra on w ruletkę, w każdej rozgrywce obstawiając 100 dolarów na czarne. W ten sposób z prawdopodobieństwem ${1\over 2}$ wygrywa kolejne 100 dolarów i z prawdopodobieństwem ${1\over 2}$ traci obstawioną kwotę. Gracz uczestniczy w 100 kolejnych rozgrywkach, chyba, że wcześniej straci wszystkie pieniądze - wtedy kończy grę.

* *Pytanie 1:*  Jak często zdarza się, że gracz przegrywa wszystkie pieniądze?
* *Pytanie 2:*   Jak często zdarza się, że gracz, wychodząc z kasyna, ma co najmniej 2000 dolarów?
* *Pytanie 3:* Jaka jest średnia kwota, z jaką gracz opuszcza kasyno?

* *Odpowiedź praktyczna*: przeprowadzimy symulacje tysiąca pobytów gracza w kasynie. Dla każdego pobytu wyznaczymy ciąg kwot, jakimi gracz dysponuje po kolejnych rozgrywkach. Zrobimy to na dwa sposoby.


# Sposób pierwszy: ciągi z wieloma zerami na końcu

Ponieważ wiemy, że hazardzista nigdy nie uczestniczy w więcej niż stu rozgrywkach, możemy zakładać, że
uczestniczy on zawsze w dokładnie stu rozgrywkach, ale kapitał nie zmienia się po pierwszym osiągnięciu
zera. Będziemy więc tworzyć ciągi stuelementowe, które mogą być zakończone wieloma zerami. W tym celu
wykonamy następujące trzy kroki:
1. Nie przejmując się tym, czy gracz faktycznie uczestniczy w stu rozgrywkach, losujemy ich wyniki
(wygrana / przegrana).
2. Przekształcamy każdy z tych wyników na wygraną kwotę: wygraną na 100, a przegraną na −100, a
następnie wyznaczamy kapitał gracza po kolejnych rozgrywkach.
3. Dla każdego pobytu w kasynie sprawdzamy, czy kapitał gracza osiągnął w którymś momencie zero.
Jeśli tak, to wszystkie wartości do końca pobytu w kasynie zamieniamy na zero.

In [14]:
import numpy as np

# Ustawienia początkowe
kapital_poczatkowy = 1000
liczba_rozgrywek = 100
liczba_wizyt = 1000

# Losujemy wyniki wszystkich rozgrywek podczas wszystkich pobytów w kasynie
wyniki_rozgrywek = np.random.binomial(1, 0.5, liczba_wizyt * liczba_rozgrywek)

# Tworzymy tabelę wyników rozgrywek: jedna kolumna to jedna wizyta w kasynie
tabela_wynikow = wyniki_rozgrywek.reshape((liczba_rozgrywek, liczba_wizyt))

# Zamieniamy zera na -100 i jedynki na 100
tabela_wygranych = 200 * tabela_wynikow - 100

# Tworzymy ciąg hipotetycznych kapitałów gracza, gdyby mógł grać na kredyt (z ujemnym kapitałem)
tabela_hipotetycznych_kapitalow = np.cumsum(tabela_wygranych, axis=0) + kapital_poczatkowy



Po pierwszym spadku do zera kapitał gracza pozostaje zerem już do końca pobytu w kasynie. Tworzymy funkcję, która dla każdej wizyty w kasynie znajduje pierwszy moment, w którym kapitał spadł do zera (o ile
taki moment istnieje) i wszystkie kolejne wartości zastępuje zerami.

In [15]:
# Definiujemy funkcję, która zamienia wszystkie wartości po pierwszym zerze (włącznie) na zera
def zamien_na_zera(kapitaly):
    gdzie_zera = np.where(kapitaly <= 0)[0]
    if gdzie_zera.size > 0:
        pierwsze_zero = gdzie_zera[0]
        kapitaly[pierwsze_zero:] = 0
    return kapitaly

# Aplikujemy funkcję na każdej kolumnie tabeli hipotetycznych kapitałów
tabela_kapitalow = np.apply_along_axis(zamien_na_zera, 0, tabela_hipotetycznych_kapitalow)


# Sposób drugi: ciągi urywające się po pierwszym zerze

Aby uzyskać ciągi, których długość odpowiada liczbie faktycznie rozegranych rund, utworzymy funkcję. Będzie ona iteracyjnie tworzyła kolejne elementy ciągu kapitałów, za każdym razem sprawdzając dwa warunki: czy liczba rozgrywek nie przekroczyła stu i czy kapitał nie spadł do zera.

In [16]:
def losuj_kapitaly(kwota, liczba_rund):
    kapital = kwota  # podaną w argumencie kwotę traktujemy jako początkowy kapitał gracza
    ciag_kapitalow = []  # do tego ciągu iteracyjnie będziemy dodawać kolejne elementy

    for runda in range(liczba_rund):  # jako sytuację domyślną traktujemy rozegranie 100 rund
        kapital += np.random.choice([-100, 100])  # dodajemy wygraną w i-tej rozgrywce
        ciag_kapitalow.append(kapital)  # dodajemy kolejny element do ciągu kapitałów

        if kapital <= 0:  # gdy gracz zbankrutuje, urywamy ciąg
            break

    return ciag_kapitalow  # wynikiem jest ciąg kapitałów dla pojedynczej wizyty w kasynie

# Teraz używamy funkcji `losuj_kapitaly` wielokrotnie, aby symulować wiele wizyt w kasynie
zestawienie = [losuj_kapitaly(kapital_poczatkowy, liczba_rozgrywek) for _ in range(liczba_wizyt)]


# Ciąg dalszy: odpowiedzi na pytania
Tworzymy ciąg składający się z ostatnich elementów ciągów kapitałów. Są to dokładnie kwoty, z jakimi gracz wychodzi z kasyna po kolejnych seriach rozgrywek.

In [17]:
import numpy as np

# Przykładowe dane dla pierwszej metody
# Załóżmy, że tabela_kapitalow jest tablicą NumPy o wymiarach (liczba_rozgrywek, liczba_wizyt)
# gdzie każda kolumna to seria kapitałów dla danej wizyty
# tabela_kapitalow = np.array([...])

# Pierwsza metoda - używając ostatniego wiersza z tabela_kapitalow
koncowe_kapitaly_1 = tabela_kapitalow[-1, :]

# Druga metoda - używając ostatniego elementu z każdej listy w zestawienie
koncowe_kapitaly_2 = np.array([symulacja[-1] if symulacja else 0 for symulacja in zestawienie])

# Obliczanie częstotliwości bankructwa dla obu metod
czestotliwosc_bankructwa_1 = np.sum(koncowe_kapitaly_1 == 0) / liczba_wizyt
czestotliwosc_bankructwa_2 = np.sum(koncowe_kapitaly_2 == 0) / liczba_wizyt

print(f"Częstotliwość bankructwa: {czestotliwosc_bankructwa_1} (sposób 1) oraz {czestotliwosc_bankructwa_2} (sposób 2)")




Częstotliwość bankructwa: 0.326 (sposób 1) oraz 0.331 (sposób 2)


In [18]:
# Obliczanie częstotliwości podwajania kapitału dla obu metod
czestotliwosc_podwajania_1 = np.sum(koncowe_kapitaly_1 >= 2000) / liczba_wizyt
czestotliwosc_podwajania_2 = np.sum(koncowe_kapitaly_2 >= 2000) / liczba_wizyt

print(f"Częstotliwość podwajania kapitału: {czestotliwosc_podwajania_1} (sposób 1) oraz {czestotliwosc_podwajania_2} (sposób 2)")


Częstotliwość podwajania kapitału: 0.197 (sposób 1) oraz 0.181 (sposób 2)


In [20]:
# Obliczanie średniego końcowego kapitału dla obu metod
sredni_koncowy_kapital_1 = np.mean(koncowe_kapitaly_1)
sredni_koncowy_kapital_2 = np.mean(koncowe_kapitaly_2)

print(f"Średni końcowy kapitał w dolarach: {sredni_koncowy_kapital_1:.2f} (sposób 1) oraz {sredni_koncowy_kapital_2:.2f} (sposób 2)")


Średni końcowy kapitał w dolarach: 1011.40 (sposób 1) oraz 982.40 (sposób 2)


# Ćwiczenie: ruina gracza

Rozważmy grę opartą o rzuty niesymetryczną monetą, dla której prawdopodobieństwo uzyskania orła wynosi $p\in(0,1)$. W grze uczestniczą dwaj gracze: pierwszy zaczyna z kapitałem $N$ zł, a drugi z kapitałem $M - N$
zł, gdzie przez $M$ oznaczamy łączny kapitał w grze. Gdy wypadnie reszka, pierwszy gracz przekazuje 1 zł
drugiemu. Gdy wypadnie orzeł, drugi gracz oddaje 1 zł pierwszemu. Gra kończy się wtedy, gdy jeden z
graczy - przegrany - straci wszystkie pieniądze na rzecz drugiego - zwycięzcy.

* *Pytanie 1*: Jak często wygrywa pierwszy gracz?
* *Pytanie 2*: Jaka jest średnia liczba rzutów monetą w czasie jednej rozgrywki?

Na powyższe pytania odpowiedz dla $p = {12\over 25}$, $M = 50$
oraz $N \in\{5, 10, 15, 20, 25, 30, 35, 40, 45\}$. Wyestymowane
prawdopodobieństwo wygranej pierwszego gracza porównaj na wykresie z prawdziwym prawdopodobieństwem, które dla $p \neq {1\over 2} wynosi


$$\rho(N)=
 {1-\left({1-p\over p}\right)^N\over 1-\left({1-p\over p}\right)^M}
   $$




* *Pytanie 3*: Dla $N \in \{5, 25, 35\}$ przedstaw przebieg pierwszych 30 wysymulowanych rozgrywek na wykresie liniowym (na osi $X$ liczba wykonanych rzutów, a na osi $Y$ kapitał pierwszego gracza). Jak dobrać skalę na osi $X$? Porównaj otrzymane wykresy.